In [1]:
#교차 검증을 그림으로 설명하기	Ch.05(05-3) 앙상블 모델 손코딩 코랩 화면 인증하기
import pandas as pd
wine = pd.read_csv('hg-mldl-master/wine.csv')
wine.head()

,alcohol,sugar,pH,class
0,9.4,1.9,3.51,0.0
1,9.8,2.6,3.20,0.0
2,9.8,2.3,3.26,0.0
3,9.8,1.9,3.16,0.0
4,9.4,1.9,3.51,0.0


In [2]:
data = wine[['alcohol', 'sugar', 'pH']].to_numpy()
target = wine['class'].to_numpy()

print(data[:10])
print(target[:10])


[[ 9.4   1.9   3.51]
 [ 9.8   2.6   3.2 ]
 [ 9.8   2.3   3.26]
 [ 9.8   1.9   3.16]
 [ 9.4   1.9   3.51]
 [ 9.4   1.8   3.51]
 [ 9.4   1.6   3.3 ]
 [10.    1.2   3.39]
 [ 9.5   2.    3.36]
 [10.5   6.1   3.35]]
[0. 0. 0. 0. 0. 0. 0. 0. 0. 0.]


In [3]:
from sklearn.model_selection import train_test_split

trainInput, testIntput, trainTarget, testTarget = train_test_split(data, target, test_size=0.2, random_state=42)
print(trainInput[:5])
print(trainTarget[:5])

[[10.5   7.7   3.19]
 [12.4   2.1   3.26]
 [11.8   2.1   3.41]
 [11.7   1.2   3.37]
 [ 9.3  15.4   3.11]]
[1. 0. 0. 1. 1.]


In [4]:
subInput, valInput, subTarget, valTarget = train_test_split(trainInput, trainTarget, test_size=0.2, random_state=42)
subInput

array([[11.4 ,  1.25,  2.95],
       [ 9.1 ,  8.  ,  3.23],
       [ 9.1 , 13.4 ,  3.06],
       ...,
       [11.5 ,  3.  ,  3.16],
       [12.3 ,  9.  ,  3.13],
       [11.1 ,  1.4 ,  3.34]])

In [5]:
from sklearn.tree import DecisionTreeClassifier
dt=DecisionTreeClassifier(random_state=42)
dt.fit(subInput, subTarget)
print(dt.score(subInput, subTarget))
print(dt.score(valInput, valTarget))

0.9971133028626413
0.864423076923077


In [6]:
from sklearn.model_selection import cross_validate
score = cross_validate(dt, trainInput, trainTarget)
score

{'fit_time': array([0.00699377, 0.00600171, 0.00699949, 0.00599957, 0.00500154]),
 'score_time': array([0.00099993, 0.        , 0.        , 0.        , 0.00099897]),
 'test_score': array([0.86923077, 0.84615385, 0.87680462, 0.84889317, 0.83541867])}

In [8]:
import numpy as np

In [9]:
from sklearn.model_selection import StratifiedKFold
scores = cross_validate(dt, trainInput, trainTarget, cv=StratifiedKFold())
np.mean(scores['test_score'])

0.855300214703487

In [10]:
splitter = StratifiedKFold(n_splits=10, shuffle=True, random_state=42)
scores = cross_validate(dt, trainInput, trainTarget, cv=splitter)
np.mean(scores['test_score'])

0.8574181117533719

In [13]:
from sklearn.model_selection import GridSearchCV
params = {'min_impurity_decrease':[0.0001, 0.0002, 0.0003, 0.0004, 0.0005]}
gs = GridSearchCV(DecisionTreeClassifier(random_state=42), params, n_jobs=-1)
gs.fit(trainInput, trainTarget)
dt = gs.best_estimator_
dt.score(trainInput, trainTarget)

0.9615162593804117

In [14]:
gs.best_params_

{'min_impurity_decrease': 0.0001}

In [15]:
gs.cv_results_

#테스트한 결과는 mean_test_screo안에

{'mean_fit_time': array([0.0066021 , 0.00659823, 0.00979862, 0.00479932, 0.00400062]),
 'std_fit_time': array([4.92853807e-04, 1.35489909e-03, 8.40085690e-03, 1.16568203e-03,
        2.02529531e-06]),
 'mean_score_time': array([0.00059834, 0.00039988, 0.00080109, 0.00059996, 0.0001997 ]),
 'std_score_time': array([0.00048855, 0.00048975, 0.00040054, 0.00048986, 0.0003994 ]),
 'param_min_impurity_decrease': masked_array(data=[0.0001, 0.0002, 0.0003, 0.0004, 0.0005],
              mask=[False, False, False, False, False],
        fill_value='?',
             dtype=object),
 'params': [{'min_impurity_decrease': 0.0001},
  {'min_impurity_decrease': 0.0002},
  {'min_impurity_decrease': 0.0003},
  {'min_impurity_decrease': 0.0004},
  {'min_impurity_decrease': 0.0005}],
 'split0_test_score': array([0.86923077, 0.87115385, 0.86923077, 0.86923077, 0.86538462]),
 'split1_test_score': array([0.86826923, 0.86346154, 0.85961538, 0.86346154, 0.86923077]),
 'split2_test_score': array([0.8825794 , 0.8

In [16]:
params = {'min_impurity_decrease':np.arange(0.0001, 0.001, 0.0001),
         'max_depth':range(5,20,1),
         'min_samples_split':range(2,100,10)}

params

{'min_impurity_decrease': array([0.0001, 0.0002, 0.0003, 0.0004, 0.0005, 0.0006, 0.0007, 0.0008,
        0.0009]),
 'max_depth': range(5, 20),
 'min_samples_split': range(2, 100, 10)}

In [17]:
gs = GridSearchCV(DecisionTreeClassifier(random_state=42), params, n_jobs=-1)
gs.fit(trainInput, trainTarget)
gs.best_params_

{'max_depth': 14, 'min_impurity_decrease': 0.0004, 'min_samples_split': 12}

In [18]:
np.max(gs.cv_results_)

{'mean_fit_time': array([0.00359893, 0.0039999 , 0.0049993 , ..., 0.0038003 , 0.00380001,
        0.0033989 ]),
 'std_fit_time': array([0.00048991, 0.00063226, 0.00089463, ..., 0.00074773, 0.00074835,
        0.00048998]),
 'mean_score_time': array([0.0006001 , 0.00099974, 0.00100093, ..., 0.00079961, 0.00059996,
        0.00040026]),
 'std_score_time': array([4.89979613e-04, 8.31393994e-07, 6.33013036e-04, ...,
        3.99806287e-04, 4.89862604e-04, 4.90212843e-04]),
 'param_max_depth': masked_array(data=[5, 5, 5, ..., 19, 19, 19],
              mask=[False, False, False, ..., False, False, False],
        fill_value='?',
             dtype=object),
 'param_min_impurity_decrease': masked_array(data=[0.0001, 0.0001, 0.0001, ..., 0.0009000000000000001,
                    0.0009000000000000001, 0.0009000000000000001],
              mask=[False, False, False, ..., False, False, False],
        fill_value='?',
             dtype=object),
 'param_min_samples_split': masked_array(data=[2, 

In [24]:
from scipy.stats import uniform, randint
rgen = randint(0, 10)
rgen.rvs(10)

array([3, 4, 3, 6, 1, 0, 7, 0, 1, 8])

In [25]:
np.unique(rgen.rvs(1000), return_counts=True)

(array([0, 1, 2, 3, 4, 5, 6, 7, 8, 9]),
 array([105,  96,  98, 104, 101,  98, 104, 108,  95,  91], dtype=int64))

In [26]:
ugen = uniform(0, 1)
ugen.rvs(10)

array([0.00220587, 0.93949287, 0.21856474, 0.80806145, 0.27152161,
       0.12405041, 0.28265551, 0.21718202, 0.9481192 , 0.1831337 ])

In [27]:
params = {'min_impurity_decrease':uniform(0.0001, 0.001),
         'max_depth':randint(20, 50),
         'min_samples_split':randint(2,25),
         'min_samples_leaf':randint(1,25),}

params

{'min_impurity_decrease': <scipy.stats._distn_infrastructure.rv_frozen at 0x1fb9bd57080>,
 'max_depth': <scipy.stats._distn_infrastructure.rv_frozen at 0x1fb9bd57940>,
 'min_samples_split': <scipy.stats._distn_infrastructure.rv_frozen at 0x1fb9bd57630>,
 'min_samples_leaf': <scipy.stats._distn_infrastructure.rv_frozen at 0x1fb9bd577b8>}

In [28]:
from sklearn.model_selection import RandomizedSearchCV
gs = RandomizedSearchCV(DecisionTreeClassifier(random_state=42), params, n_iter=100, n_jobs=-1, random_state=42)
gs.fit(trainInput, trainTarget)
gs.best_params_

{'max_depth': 39,
 'min_impurity_decrease': 0.00034102546602601173,
 'min_samples_leaf': 7,
 'min_samples_split': 13}

In [29]:
np.max(gs.cv_results_)

{'mean_fit_time': array([0.00459852, 0.00479836, 0.00459871, 0.0057991 , 0.00560002,
        0.00760031, 0.0058001 , 0.00339899, 0.00579958, 0.00379934,
        0.00379925, 0.00319905, 0.00339928, 0.00699944, 0.00359912,
        0.0037972 , 0.00339804, 0.00380039, 0.00379896, 0.00919995,
        0.00719872, 0.00360036, 0.00379968, 0.00419946, 0.00379977,
        0.00539904, 0.00339899, 0.00319972, 0.00379877, 0.00360031,
        0.00459838, 0.0031991 , 0.00779967, 0.00379958, 0.00399938,
        0.00340047, 0.00459919, 0.00340066, 0.00379949, 0.00379949,
        0.00379882, 0.00400085, 0.00519962, 0.00519934, 0.00400009,
        0.00339956, 0.00459938, 0.0036005 , 0.00599961, 0.00379696,
        0.00379972, 0.00479951, 0.00519962, 0.00439963, 0.00380001,
        0.00519972, 0.00439987, 0.00400043, 0.00299883, 0.00339952,
        0.0073998 , 0.00380015, 0.00360041, 0.00379939, 0.00379977,
        0.00340009, 0.00340066, 0.0037992 , 0.00339937, 0.00339904,
        0.00380011, 0.00399961,